In [64]:
import requests
from bs4 import BeautifulSoup
import os
from PIL import Image, ImageOps
import csv
import base64

In [51]:
response = requests.get('https://pokemondb.net/pokedex/all')
soup = BeautifulSoup(response.text, 'html.parser')
rows = soup.select('table#pokedex tbody tr')
pokemon_links = []
for row in rows:
  link_tag = row.find('a', href=True) # gets links
  if link_tag:
    pokemon_links.append('https://pokemondb.net' + link_tag['href'])

pokemon_links = list(sorted(set(pokemon_links)))
print(len(pokemon_links))


1025


In [ ]:
import csv

os.makedirs('pokemon_images', exist_ok=True)

# resize func
def resize_image_to_512(img_path, output_path):
    with Image.open(img_path) as img:
        img.thumbnail((512, 512))
        canvas = Image.new('RGB', (512, 512), (255, 255, 255))
        paste_x = (512 - img.width) // 2
        paste_y = (512 - img.height) // 2
        canvas.paste(img, (paste_x, paste_y))
        canvas.save(output_path)

# CSV file setup
csv_filename = 'pokemon_data.csv'
csv_headers = ['Name', 'Image Path', 'Type 1', 'Type 2']
with open(csv_filename, mode='w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(csv_headers)

    # gets info + downloads image + writes to CSV for all pokemon
    for url in pokemon_links:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # 1. name
        name = soup.select_one('h1').text.strip()

        # 2. img urls
        img_url = f"https://img.pokemondb.net/artwork/{name.lower()}.jpg"

        # 3. types
        rows = soup.select('tr')
        second_row = rows[1] if len(rows) > 1 else None
        type_links = second_row.select('a') if second_row else None
        type1 = type_links[0].text.strip() if len(type_links) > 0 else None
        type2 = type_links[1].text.strip() if len(type_links) > 1 else None

        # download & resize
        try:
            img_response = requests.get(img_url)
            img_response.raise_for_status()
            temp_img_path = f'pokemon_images/{name}_original.jpg'
            final_img_path = f'pokemon_images/{name}.jpg'

            # saves image
            with open(temp_img_path, 'wb') as img_file:
                img_file.write(img_response.content)
            resize_image_to_512(temp_img_path, final_img_path)
            os.remove(temp_img_path)

            # create CSV
            csvwriter.writerow([name, final_img_path, type1, type2])
            print(f"Processed {name}: {final_img_path}, {type1}, {type2}")
        except requests.exceptions.RequestException as e:
            print(f"Failed to download image for {name}: {e}")
        except Exception as e:
            print(f"Error processing data for {name}: {e}")

#print(f"Data exported to {csv_filename}")
